In [1]:
data Exp = Num Int
         | Exp :+: Exp
         | Exp :/: Exp
         | Var String

In [20]:
newtype Env a b = Env (a -> Maybe b)
newtype ExprEnv = EE (Env String Int)

add :: Eq a => a -> b -> Env a b -> Env a b
add k v (Env env) = Env $ \k' -> if k == k' then Just v else env k'

```haskell
evalWithEnv :: ExprEnv -> Exp -> Maybe Int
evalWithEnv = undefined

- Die Funktion mit Applicative muss mit `seqApp` also `<*>` implementiert werden
- Die Funktion für Monad hingegen mit `bind`, also `>>=`

```haskell
seqAp :: f (a -> b) -> f a -> f b
bind :: m a -> (a -> m b) -> m b

# Funktion um aus einem Env object ein Maybe Objekt zu machen

In [25]:
matchEE :: Env String Int -> String -> Maybe Int
matchEE (Env env) = env

# Funktion mit bind

In [37]:
evalWithEnv :: ExprEnv -> Exp -> Maybe Int
evalWithEnv (EE ee) (Num e) = Just e
evalWithEnv (EE ee) (Var e) = matchEE ee e
evalWithEnv (EE ee) (l :+: r) = do 
    ll <- evalWithEnv (EE ee) l
    rr <- evalWithEnv (EE ee) r
    return (ll + rr)
evalWithEnv (EE ee) (l :/: r) = do 
    ll <- evalWithEnv (EE ee) l
    case ll of
        0 -> Nothing
        other -> do
            rr <- evalWithEnv (EE ee) r
            return (ll `div` rr)

In [39]:
evalWithEnv :: ExprEnv -> Exp -> Maybe Int
evalWithEnv (EE ee) (Num e) = Just e
evalWithEnv (EE ee) (Var e) = matchEE ee e
evalWithEnv (EE ee) (l :+: r) = evalWithEnv (EE ee) l >>= \ll -> evalWithEnv (EE ee) r >>= \rr -> return (ll + rr)
evalWithEnv (EE ee) (l :/: r) = evalWithEnv (EE ee) l >>= \ll -> if ll == 0 then Nothing else evalWithEnv (EE ee) r >>= \rr -> return (ll `div` rr)

# Funktion mit pure

```haskell 
(pure (+) <*> eval e1) <*> eval e2
(+) <$> eval e1 <*> eval e2

pure (flip div) <*> mv2 <*> eval e1
(flip div) <$> mv2 <*> eval e1

In [45]:
evalWithEnv :: ExprEnv -> Exp -> Maybe Int
evalWithEnv (EE ee) (Num e) = Just e
evalWithEnv (EE ee) (Var e) = matchEE ee e
evalWithEnv (EE ee) (l :+: r) = (+) <$> evalWithEnv (EE ee) l <*> evalWithEnv (EE ee) r
evalWithEnv (EE ee) (l :/: r) = flip div <$> evalWithEnv (EE ee) r <*> if evalWithEnv (EE ee) l == Just 0 then Nothing else evalWithEnv (EE ee) l 

# Funktor instanzen

In [48]:
:k Exp

Exp :: *

Da Exp vom Kind * ist, kann keine Funktor instanz davon erstellt werden

In [51]:
:k Env Int

Env Int :: * -> *

Env a ist vom Kind * -> *, das heisst dies sollte möglich sein

In [108]:
-- DAS HIER GEHT NICHT, DA ICH DEN WERT AUS DEM JUST RAUSHOLEN MUSS. UND DAS GEHT NUNMAL NUR MIT DEM CASE
instance Functor (Env a) where 
    -- b :: a
    -- a :: a -> Maybe b
    -- f :: b -> c
    fmap :: (b -> c) -> Env a b -> Env a c
    fmap f (Env a) = Env (\b -> Just $ f $ a b)

Line 6: Avoid lambda
Found:
\ b -> Just $ f $ a b
Why not:
Just . f . a

: 

In [105]:
instance Functor (Env a) where 
    -- b :: a
    -- a :: a -> Maybe b
    -- f :: b -> c
    fmap :: (b -> c) -> Env a b -> Env a c
    fmap f (Env a) = Env (\b -> 
        case a b of 
            Just x -> Just $ f x
            other -> Nothing)